<a href="https://colab.research.google.com/github/Yash-invic/Architecting-LLMs-WiDS/blob/main/week1/week1_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###micrograd exercises

## section 1: derivatives

In [ ]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

In [ ]:
# write the function df that returns the analytical gradient of f
def gradf(a, b, c):
  dfda = -3*a**2 - 0.5*a**(-0.5)
  dfdb =3*cos(3*b) + 2.5*b**1.5
  dfdc = c**(-2)
  return [dfda, dfdb, dfdc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


In [ ]:
# now estimate the gradient numerically without any calculus, using

# -----------
numerical_grad = [0, 0, 0] # TODO
# -----------
def num_grad(a, b, c):
  h = 1e-6
  dfda1 = (f(a+h, b, c) - f(a, b, c))/h
  dfdb1 = (f(a, b+h, c) - f(a, b, c))/h
  dfdc1 = (f(a, b, c+h) - f(a, b, c))/h
  return [dfda1, dfdb1, dfdc1]
numerical_grad = num_grad(2, 3, 4)

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


In [ ]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.

# -----------
numerical_grad2 = [0, 0, 0] # TODO
# -----------
def num_grad2(a, b, c):
  h1 = 1e-5
  dfda2 = (f(a+h1, b, c) - f(a-h1, b, c))/(2*h1)
  dfdb2 = (f(a, b+h1, c) - f(a, b-h1, c))/(2*h1)
  dfdc2 = (f(a, b, c+h1) - f(a, b, c-h1))/(2*h1)
  return [dfda2, dfdb2, dfdc2]
numerical_grad2 = num_grad2(2, 3, 4)


for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


## section 2: support for softmax

In [ ]:
# Value class starter code, with many functions taken out
from math import exp, log
class Value:

  def __init__(self, data, _children=(), _op = ''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op= _op

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += out.grad
      other.grad += out.grad
    out._backward = _backward

    return out

  # ------
  # re-implement all the other functions needed for the exercises below
  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data*other.data, (self, other), '*')
    def _backward():
      self.grad += other.data*out.grad
      other.grad += self.data*out.grad
    out._backward = _backward
    return out
  def __neg__(self):
    return self*-1
  def __radd__(self, other):
    return self + other
  def __sub__(self, other):
    return self + (-other)
  def __rsub__(self, other):
    return other + (-self)
  def __rmul__(self, other):
    return self*other
  def __truediv__(self, other):
    return self*(other**-1)
  def __rtruediv__(self, other):
    return other*(self**-1)
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "using only supporting powers(int/float) for now"
    out = Value(self.data**other, (self,), f'**{other}')
    def _backward():
      self.grad += (other*(self.data**(other-1)))*out.grad
    out._backward = _backward
    return out
  def exp(self):
    x = self.data
    out = Value(exp(x), (self,), 'exp')
    def _backward():
      self.grad += out.data*out.grad
    out._backward = _backward
    return out
  def relu(self):
    out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')
    def _backward():
      self.grad += (out.data > 0)*out.grad
    out._backward = _backward
    return out
  def log(self):
    x = self.data
    out = Value(log(x), (self,), 'log')
    def _backward():
      self.grad += (1/x)*out.grad
    out._backward = _backward
    return out
  # TODO
  # ------

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()


In [ ]:
# without referencing our code/video __too__ much, make this cell work
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


In [ ]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
logits = torch.tensor([0.0, 3.0, -2.0, 1.0], dtype = torch.double, requires_grad = True)
probs = torch.nn.functional.softmax(logits, dim = 0)
loss = -probs[3].log()
loss.backward()
print('PyTorch Output-\n', logits.grad)